# 04 – Preprocessing & Feature Engineering
### Car Price Prediction Using Machine Learning
Group Assignment 02 - CCS3012 - Data Analytics  
Submission Date: 16th September 2025

---

### **Group 11**
-  **FC211034 - N.D. Samararathne Kodikara**
-  **FC211013 - N.W.V. Tharindu Pabasara**
-  **FC211025 - W.M.M.C.B. Wijesundara**



---


### **Supervisor**
**Ms. Dilmi Praveena**  
*Faculty of Computing*  
*University of Sri Jayewardenepura*

---


## 📌 Objectives

This notebook builds upon the cleaned dataset produced in **Notebook 01 — Data Exploration & Cleaning**.
The focus here is to prepare the dataset for modeling by performing outlier detection, handling, and feature engineering, including:

---

- **Outlier detection & handling**: We have identified extreme and unrealistic values some numerical features such as price, mileage, levy, engine_volume and here we will decide whether to remove or retain them.

- **Feature engineering**: Create new features to better capture information.

- **Data consistency & cleaning**: Ensure all engineered features are correctly formatted, handle missing or zero values appropriately, and reduce noise in the dataset..

---

### 📂 Input  
- `clean_data.csv` saved in `Data/processed/`  
---

### 📊 Dataset Overview
**Dataset:** Car price dataset cleaned.  
**columns include**:   
- `id`, `price` (target variable),  `levy`,  `manufacturer`,  `model`,  `prod_year`,  `category`,  `leather_interior`,  `fuel_type`, `engine_volume`,  `mileage`,  `cylinders`,  `gear_box_type`,  `drive_wheels`,  `doors`,  `wheel`, `color`,  `airbags`,  `turbo` (derived in Notebook 01)


> **Dataset stats:** 19,237 rows × 19 features | Target variable: `price` | Problem type: Regression


# Setup & imports

In [8]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns


# Statistical functions
from scipy import stats

from prettytable import PrettyTable 

In [9]:
df = pd.read_csv("./Data/processed/clean_data.csv")


In [10]:
# Prints the first 5 rows of the DataFrame.
df.head()


,id,price,levy,manufacturer,model,prod_year,category,leather_interior,fuel_type,engine_volume,mileage,cylinders,gear_box_type,drive_wheels,doors,wheel,color,airbags,turbo
0,45654403,13328,1399.0,lexus,rx 450,2010,jeep,yes,hybrid,3.5,186005,6,automatic,4x4,04-may,left wheel,silver,12,False
1,44731507,16621,1018.0,chevrolet,equinox,2011,jeep,no,petrol,3.0,192000,6,tiptronic,4x4,04-may,left wheel,black,8,False
2,45774419,8467,0.0,honda,fit,2006,hatchback,no,petrol,1.3,200000,4,variator,front,04-may,right-hand drive,black,2,False
3,45769185,3607,862.0,ford,escape,2011,jeep,yes,hybrid,2.5,168966,4,automatic,4x4,04-may,left wheel,white,0,False
4,45809263,11726,446.0,honda,fit,2014,hatchback,yes,petrol,1.3,91901,4,automatic,front,04-may,left wheel,silver,4,False


In [11]:
# Prints the name of each column in the dataset, the number of non-null values it contains, and its data type.
def df_info(df):
    table = PrettyTable()
    table.field_names = ["Column", "Non-Null Count", "Dtype"]

    for col in df.columns:
        non_null_count = df[col].count()
        dtype = df[col].dtype
        table.add_row([col, non_null_count, dtype])

    print(table)

df_info(df)

+------------------+----------------+---------+
|      Column      | Non-Null Count |  Dtype  |
+------------------+----------------+---------+
|        id        |     15697      |  int64  |
|      price       |     15697      |  int64  |
|       levy       |     15697      | float64 |
|   manufacturer   |     15697      |  object |
|      model       |     15697      |  object |
|    prod_year     |     15697      |  int64  |
|     category     |     15697      |  object |
| leather_interior |     15697      |  object |
|    fuel_type     |     15697      |  object |
|  engine_volume   |     15697      | float64 |
|     mileage      |     15697      |  int64  |
|    cylinders     |     15697      |  int64  |
|  gear_box_type   |     15697      |  object |
|   drive_wheels   |     15697      |  object |
|      doors       |     15697      |  object |
|      wheel       |     15697      |  object |
|      color       |     15697      |  object |
|     airbags      |     15697      |  i

💡 **Observations:**  
- The data set appears unchanged and matches the previously saved version.

Le't start...

# **Deterministic cleaning & derived column**
Doing before train/test split because cause no target leakage

## `doors` column

> 🧩 Standardize to two representations:
> - `doors_count` numeric: 2, 4, 6 (use 6 as proxy for >5)
> - `doors_cat` original categorical for interactions

In [5]:
def doors_to_count(s):
    if pd.isna(s): return np.nan
    s = str(s).lower()
    if '02' in s or '02-mar' in s: return 2
    if '04' in s or '04-may' in s: return 4
    if '>' in s or '>5' in s: return 6
    # fallback: try to extract first number
    import re
    m = re.search(r'\d+', s)  # fallback: extract first number in the string
    return int(m.group()) if m else np.nan

df['doors_count'] = df['doors'].apply(doors_to_count)
print(f"✅ created doors_count column.")
df['doors_count'].value_counts()


✅ created doors_count column.


doors_count
4    14835
2      738
6      124
Name: count, dtype: int64

In [7]:
# Mapping dictionary
doors_map = {
    '02-mar': '2-3',
    '04-may': '4-5',
    '>5': '>5'
}

# Apply mapping
df['doors_cat'] = df['doors'].map(doors_map).fillna(df['doors'])
print(f"✅ created doors_cat column.")
df['doors_cat'].value_counts()

✅ created doors_cat column.


doors_cat
4-5    14835
2-3      738
>5       124
Name: count, dtype: int64

## `Cylinders` column

> 🧩 grouping into buckets  
> 🧩 Ordinal encoding. start with `1`


In [9]:
def cylinder_group(x):
    if pd.isna(x): return np.nan
    if x <= 2: return '1-2'
    if 3 <= x <= 4: return '3-4'
    if 5 <= x <= 8: return '5-8'
    return '>8'

df['cylinder_group'] = df['cylinders'].apply(cylinder_group)
print(f"✅ created cylinder_group column.")
df['cylinder_group'].value_counts()


✅ created cylinder_group column.


cylinder_group
3-4    12372
5-8     3196
1-2       78
>8        51
Name: count, dtype: int64

In [12]:
# Define the order
cylinder_order = ['1-2', '3-4', '5-8', '>8']

# Map to ordinal numbers
df['cylinder_group_ord'] = df['cylinder_group'].map({k: i+1 for i, k in enumerate(cylinder_order)})

print(f"✅ created cylinder_group_ord column.\n")
print(df[['cylinder_group', 'cylinder_group_ord']].head())


✅ created cylinder_group_ord column.

  cylinder_group  cylinder_group_ord
0            5-8                   3
1            5-8                   3
2            3-4                   2
3            3-4                   2
4            3-4                   2


## `Airbags` column

> 🧩 Grouping ['0','1-2','3-4','5-8','>8']  
> 🧩 Ordinal Encoding. start with `0`

In [13]:
# Function to group airbags
def airbags_group(n):
    if pd.isna(n):  # handle missing values
        return np.nan
    if n == 0:
        return '0'
    if 1 <= n <= 2:
        return '1-2'
    if 3 <= n <= 4:
        return '3-4'
    if 5 <= n <= 8:
        return '5-8'
    return '>8'

# Apply to datset
df['airbags_group'] = df['airbags'].apply(airbags_group)
print(f"✅ created airbags_group column.")
df['airbags_group'].value_counts()


✅ created airbags_group column.


airbags_group
3-4    5141
>8     5019
5-8    3040
0      1378
1-2    1119
Name: count, dtype: int64

In [18]:
airbags_order = ['0', '1-2', '3-4', '5-8', '>8']  # define order

# ordinal encoding
df['airbags_group_ord'] = df['airbags_group'].map({k: i for i, k in enumerate(airbags_order)})

print(f"✅ created airbags_group_ord column.\n")
print(df[['airbags_group', 'airbags_group_ord']].head())


✅ created airbags_group_ord column.

  airbags_group  airbags_group_ord
0            >8                  4
1           5-8                  3
2           1-2                  1
3             0                  0
4           3-4                  2


## `Car Age` column

> 🧩 current_year - prod_year

In [20]:
import datetime

current_year = datetime.datetime.now().year  # get current year

df['car_age'] = current_year - df['prod_year']

print(f"✅ created car_age column.")
# check for negative or weird values
print(df['car_age'].describe())


✅ created car_age column.
count    15697.000000
mean        14.410015
std          5.697194
min          5.000000
25%         11.000000
50%         13.000000
75%         17.000000
max         45.000000
Name: car_age, dtype: float64


## `leather_interior` column

> 🧩 create `has_leather` binary : Directly captures the positive price effect of leather.

In [28]:
# Function map has_leather
def has_leather(x):
    if pd.isna(x):
        return 0 
    if str(x).strip().lower() in ['yes']:
        return 1
    return 0

# Apply to df
df['has_leather'] = df['leather_interior'].apply(has_leather)

print(f"✅ created has_leather column.\n")

# Quick check
print(df[['leather_interior', 'has_leather']].value_counts())


✅ created has_leather column.

leather_interior  has_leather
yes               1              10547
no                0               5150
Name: count, dtype: int64
